In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
import math
from nltk.corpus import stopwords

# COMPUTE TF E TFIDF MY WAY, GROUPED AND NOT GROUPED 
look at the columns after computeData, everything is there

In [2]:
#apply description pre processing

#arguments {stemming = [True, False]}
def preProcess(stemming):
    if(stemming):
        data = pd.read_csv("StemmedWord2vecTop3.csv")
    elif( not stemming):
        print("we're working only on stemmed for now,change stemming = True")
        #data = pd.read_csv("dataAfterPOS.csv") #generate this through Notebook1 
    data.drop(["region_2"],inplace=True,axis=1)
    #remove stopwords
    stop = stopwords.words('english')

    data["stopwords"] = data["description"].apply(lambda x: len([x for x in x.split() if x in stop]))
    
    #remove commas, fullstops...
    data['description'] = data['description'].str.replace('[^\w\s]','')
    
    #remove most frequent and less frequent bcs useless
    freq = pd.Series(' '.join(data['description']).split()).value_counts()[:10]
    freq = list(freq.index)
    data['description'] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

    unfreq = pd.Series(' '.join(data["description"]).split()).value_counts()
    count = 0
    for el in unfreq: 
        if(el < 10):
            count = count + 1

    unfreqList = unfreq[-count:]
    unfreq = dict(unfreqList)  
    data["description"] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in unfreq))

    
    try:
        data.drop(["word_count","stopwords"],axis=1, inplace=True)
    except: 
        print("ok")
    ##added word count
    data["word_count"] = data["description"].apply(lambda x: len(str(x).split(" ")))
    return data

In [3]:
def computeData(data,stemming):
            #take only the description with 3 words
            fullText = pd.Series(' '.join(data['description']).split()).value_counts()
            totalWords = pd.Series("".join(data["description"]).split()).value_counts().sum()
            print(totalWords)

            t1 = data.groupby(["variety"])
            t2 = pd.DataFrame({'text' : t1['description'].apply(lambda x: (x + " ").sum())}).reset_index()
            t2["word_count"] = t2["text"].apply(lambda x: len(str(x).split(" ")))
            t2 = t2.sort_values('word_count',ascending = False)
            #t2.iloc[0]["text"]
            #t2
            varietyDict = {}
            for idx, row in t2.iterrows():
                varietyDict[row["variety"]] = pd.Series(row['text'].split()).value_counts()
            tfDict = {}
            for var in varietyDict:
                temp = {}
                for word in varietyDict[var].keys():
                    temp[word] = varietyDict[var][word] / fullText[word]   #compute tf= numero di occorrenze della parola nel documento /numero di occorrenze della stessa parola in tutti i documenti 
                tfDict[var] = temp
            #tfDict["Pinot Gris"] 
            
            desc = []
            for idx, row in data.iterrows():
                modded = []
                temp = row["description"].split()
                for word in temp:
                    modded.append(tfDict[row["variety"]][word])
                desc.append(modded)

            data["tf_grouped"] = desc  #here you can add as a column the results for term frequency

            
            varList = data["variety"].unique().tolist()

            def computeIdf1(dataset):
                a = []
                idf = {}
                for idx,row in dataset.iterrows():
                    el = row["variety"]
                    docList = data[data["variety"] == el]
                    N = len(docList)
                    idfDict = {}
                    temp = t2[t2["variety"] == el]
                    idfDict = dict.fromkeys(temp.iloc[0]["text"].split(),0)

                    for idx,row in docList.iterrows():
                        for word in row["description"].split():
                            idfDict[word] += 1

                    for word, val in idfDict.items():
                        idfDict[word] = math.log10(N/float(val))
                    a.append([el,N,idfDict])    
                    idf[el] = idfDict
                return idf, a

            idf,a = computeIdf1(t2)

            tfIdf = {}
            for var in varietyDict:
                temp = {}
                for word in tfDict[var].keys():
                    temp[word] = tfDict[var][word] * idf[var][word]
                tfIdf[var] = temp 

            desc1 = []
            for idx, row in data.iterrows():
                modded = []
                temp = row["description"].split()
                for word in temp:
                    modded.append(tfIdf[row["variety"]][word])
                desc1.append(modded)

            data["tfIdf_grouped"] = desc1
            #get word count
            data["word_count"] = data["description"].apply(lambda x: len(str(x).split(" ")))
            
            try:
                data.drop(["stopwords"],axis=1, inplace=True)
            except: 
                print("no stopwords found (it's ok don't worry)")
            
            #if stemming:
             #   data.to_csv("stemmedData_tfIdf_groupedByVariety.csv", encoding='utf-8', index=False)
            #else: 
             #   data.to_csv("notStemmedData_tfIdf_groupedByVariety.csv", encoding='utf-8', index=False)
    #if we want to work on the sum of the descriptions as a whole document
            fullText = pd.Series(' '.join(data['description']).split()).value_counts()
            #getValueCounts
            tfData = {}
            for idx, row in data.iterrows(): 
                tfData[idx] = pd.Series(row['description'].split()).value_counts();

            #compute tf for every word
            tf1 = {}
            for idx in tfData.keys():
                tf1[idx] = {}
                for word in tfData[idx].keys():
                    tf1[idx][word] = tfData[idx][word] / fullText[word]

            desc = []
            for idx, row in data.iterrows():
                modded = []
                temp = row["description"].split()
                for word in temp:
                    modded.append(tf1[idx][word])
                desc.append(modded)

            data["tf_fullData"] = desc  #here you can add as a column the results for term frequency
            
            #idf
            docCount = dict.fromkeys(fullText.keys(),0)
            idfDict1 = {}
            N = len(data)                                #numero di documenti #dizionario contenente tutte le parole presenti 
            idfData = dict(tfData)
            for el in idfData:     #per ogni descrizione
                for word,val in idfData[el].items():
                    docCount[word] = docCount[word] + 1

            for word in fullText.keys():
                idfDict1[word] = math.log10(N/docCount[word])  

            #compute tf-idf
            tfIdf1 = {}
            for idx in tfData.keys():
                tfIdf1[idx] = {}
                for word in tfData[idx].keys():
                    val = tf1[idx][word] * idfDict1[word]
                    tfIdf1[idx][word] = val
            #tfIdf1[0]

            desc1 = []
            for idx, row in data.iterrows():
                modded = []
                temp = row["description"].split()
                for word in temp:
                    modded.append(tfIdf1[idx][word])
                desc1.append(modded)
            #desc1[0]
            data["tfIdf_fullData"] = desc1
            #get word count
            data["word_count"] = data["description"].apply(lambda x: len(str(x).split(" ")))
            
            try:
                data.drop(["stopwords"],axis=1, inplace=True)
            except: 
                print("no stopwords found (it's ok don't worry)")
            
            #if stemming:
             #   data.to_csv("stemmedData_tf_tfIdf_fullDataset.csv", encoding='utf-8', index=False)
            #else: 
            #    data.to_csv("notStemmedData_tf_tfIdf_fullDataset.csv", encoding='utf-8', index=False)

In [5]:
stemming = True #if True apply on stemming 
data = preProcess(stemming)
data

ok


,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,similarityTop3WinesByVariety,stopwords,word_count
0,Italy,tropic broom brimston herb express appl citru ...,Vulkà Bianco,87,17.000000,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,1.142944,0,10
1,Portugal,fruiti firm juici red berri drinkabl,Avidagos,87,15.000000,Douro,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,1.078157,0,6
2,US,tart snappi lime flesh domin green pineappl po...,unknown,87,14.000000,Oregon,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,1.156945,0,10
3,US,pineappl rind lemon pith orang blossom opul no...,Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,1.025862,0,13
4,US,much regular bottl rough tannic rustic earthi ...,Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,1.192760,0,16
5,Spain,blackberri typic navarran whiff green herb cas...,Ars In Vitro,87,15.000000,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,0.648527,0,17
6,Italy,bright inform candi berri white pepper savori ...,Belsito,87,16.000000,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,0.963839,0,10
7,France,dri restrain offer profus firm textur much food,unknown,87,24.000000,Alsace,Alsace,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,1.094184,0,8
8,Germany,savori thyme note accent preserv peach brisk o...,Shine,87,12.000000,Rheinhessen,unknown,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,1.050787,0,12
9,France,great depth fresh appl pear touch spice dri cr...,Les Natures,87,27.000000,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,1.043785,0,11


In [5]:
len(data)

117652

In [6]:
len(data)

117652

In [7]:
#groupByVariety = False #if true apply grouping by variety for the description analysis
computeData(data,stemming)

1752815
no stopwords found (it's ok don't worry)


In [8]:
data.head(2)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,similarityTop3WinesByVariety,word_count,tf_grouped,tfIdf_grouped,tf_fullData,tfIdf_fullData
0,Italy,tropic broom brimston herb express appl citru ...,Vulkà Bianco,87,17.0,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,1.142944,10,"[0.0419835716458777, 0.16470588235294117, 0.16...","[0.04885833490039738, 0.3057715203598046, 0.38...","[0.000304228780042592, 0.0058823529411764705, ...","[0.0004757190717126231, 0.016706767017015508, ..."
1,Portugal,fruiti firm juici red berri drinkabl,Avidagos,87,15.0,Douro,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,1.078157,6,"[0.05599777860051833, 0.05929317834918398, 0.0...","[0.03228609409338993, 0.03883847038764645, 0.0...","[9.255831173639393e-05, 0.00011741223435481978...","[9.783838242588247e-05, 0.00013509834598890726..."


# Take the top 3 of each list

In [9]:
a = pd.DataFrame
a = data

In [13]:
import ast
a.head(2)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,similarityTop3WinesByVariety,word_count,tf_grouped,tfIdf_grouped,tf_fullData,tfIdf_fullData
0,Italy,tropic broom brimston herb express appl citru ...,Vulkà Bianco,87,17.0,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,1.142944,10,"[0.0419835716458777, 0.16470588235294117, 0.16...","[0.04885833490039738, 0.3057715203598046, 0.38...","[0.000304228780042592, 0.0058823529411764705, ...","[0.0004757190717126231, 0.016706767017015508, ..."
1,Portugal,fruiti firm juici red berri drinkabl,Avidagos,87,15.0,Douro,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,1.078157,6,"[0.05599777860051833, 0.05929317834918398, 0.0...","[0.03228609409338993, 0.03883847038764645, 0.0...","[9.255831173639393e-05, 0.00011741223435481978...","[9.783838242588247e-05, 0.00013509834598890726..."


# non serve più fare questa conversione perchè sono già numeri dato che non li leggiamo da file, 
#a['tf_grouped'] = a['tf_grouped'].map(ast.literal_eval)
#a['tfIdf_grouped'] = a['tfIdf_grouped'].map(ast.literal_eval)
#a['tf_fullData'] = a['tf_fullData'].map(ast.literal_eval)
#a['tfIdf_fullData'] = a['tfIdf_fullData'].map(ast.literal_eval)


In [ ]:
#check the max lenght

In [ ]:
max_tf = 0
max_tfIdf = 0
for index, row in a.iterrows(): 
       #print(len(row['description_values_tf']))
        len_row_tf = len(row['description_values_tf'])
        len_row_tfIdf = len(row['description_values_tfIdf'])
        
        if (len_row_tf > max_tf):
            max_tf = len_row_tf
            
        if (len_row_tfIdf > max_tfIdf):
            max_tfIdf = len_row_tfIdf
            
        # print(len(row[description_values_tfIdf]))


In [ ]:
#take the tre hight value and put in the colum
#

In [38]:
import heapq

def take_top3(a,col):
    
    new_1=col+'_1'
    new_2=col+'_2'
    new_3=col+'_3'
    a[new_1]=0
    a[new_2]=0
    a[new_3]=0
    
    for index, row in a.iterrows(): 
        #print(row['description_values_tf'])
        #z = np.partition(row['description_values_tf'], 2)[:3]
        x = heapq.nlargest(3, row[col])#change this value to  description_values_tfIdf to get it
        try:
            a.loc[index,new_1] = x[0]
            try:
                a.loc[index,new_2] = x[1]

                try:
                    a.loc[index,new_3] = x[2]
                    pass
                except IndexError:
                    a.loc[index,new_3] = 0
                    pass

            except IndexError:
                a.loc[index,new_2] = 0
                a.loc[index,new_3] = 0
                pass

        except IndexError:
            a.loc[index,new_1] = 0
            a.loc[index,new_2] = 0
            a.loc[index,new_3] = 0
            pass
    return a


    


    

In [39]:
a.head(1)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,similarityTop3WinesByVariety,word_count,tf_grouped,tfIdf_grouped,tf_fullData,tfIdf_fullData,new_1,new_2,new_3
0,Italy,tropic broom brimston herb express appl citru ...,Vulkà Bianco,87,17.0,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,1.142944,10,"[0.0419835716458777, 0.16470588235294117, 0.16...","[0.04885833490039738, 0.3057715203598046, 0.38...","[0.000304228780042592, 0.0058823529411764705, ...","[0.0004757190717126231, 0.016706767017015508, ...",0.164706,0.163636,0.047729


In [43]:
a = take_top3(a,'tf_grouped')
a = take_top3(a,'tfIdf_grouped')
a = take_top3(a,'tf_fullData')
a = take_top3(a,'tfIdf_fullData')

In [46]:
a.head(1)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,tf_grouped_3,tfIdf_grouped_1,tfIdf_grouped_2,tfIdf_grouped_3,tf_fullData_1,tf_fullData_2,tf_fullData_3,tfIdf_fullData_1,tfIdf_fullData_2,tfIdf_fullData_3
0,Italy,tropic broom brimston herb express appl citru ...,Vulkà Bianco,87,17.0,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,...,0.047729,0.384445,0.305772,0.048858,0.018182,0.005882,0.00064,0.06055,0.016707,0.001205


In [45]:
a= a.drop(['tf_grouped','tfIdf_grouped','tf_fullData','tfIdf_fullData'], axis=1)

In [48]:
list(a)

['country',
 'description',
 'designation',
 'points',
 'price',
 'province',
 'region_1',
 'taster_name',
 'title',
 'variety',
 'winery',
 'similarityTop3WinesByVariety',
 'word_count',
 'tf_grouped_1',
 'tf_grouped_2',
 'tf_grouped_3',
 'tfIdf_grouped_1',
 'tfIdf_grouped_2',
 'tfIdf_grouped_3',
 'tf_fullData_1',
 'tf_fullData_2',
 'tf_fullData_3',
 'tfIdf_fullData_1',
 'tfIdf_fullData_2',
 'tfIdf_fullData_3']

In [ ]:
#a.drop(["description"],inplace=True,axis=1)

In [103]:
a.to_csv("StemmedWord2vecTop3_parsed.csv", encoding='utf-8', index=False)


# match the weather data

In [104]:
wine_base= pd.read_csv("StemmedWord2vecTop3_parsed.csv", encoding='utf-8', index_col=False)


In [106]:
wine_base.head(1)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,tf_grouped_3,tfIdf_grouped_1,tfIdf_grouped_2,tfIdf_grouped_3,tf_fullData_1,tf_fullData_2,tf_fullData_3,tfIdf_fullData_1,tfIdf_fullData_2,tfIdf_fullData_3
0,Italy,tropic broom brimston herb express appl citru ...,Vulkà Bianco,87,17.0,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,...,0.047729,0.384445,0.305772,0.048858,0.018182,0.005882,0.00064,0.06055,0.016707,0.001205


In [107]:
df_rain = pd.read_csv("dataset_weather/pr_1901_2015.csv", index_col=0)
df_temp = pd.read_csv("dataset_weather/tas_1901_2015.csv", index_col=0)
df_temp = df_temp.reset_index()
df_rain = df_rain.reset_index()


In [108]:
def state_sing(df):
    df.loc[(df['Country'] == 'ITA'), 'Country'] = 'Italy'
    df.loc[(df['Country'] == 'PRT'), 'Country'] = 'Portugal'
    df.loc[(df['Country'] == 'USA'), 'Country'] = 'US'
    df.loc[(df['Country'] == 'ESP'), 'Country'] = 'Spain'
    df.loc[(df['Country'] == 'FRA'), 'Country'] = 'France'
    df.loc[(df['Country'] == 'DEU'), 'Country'] = 'Germany'
    df.loc[(df['Country'] == 'ARG'), 'Country'] = 'Argentina'
    df.loc[(df['Country'] == 'CHL'), 'Country'] = 'Chile'
    df.loc[(df['Country'] == 'AUS'), 'Country'] = 'Australia'
    df.loc[(df['Country'] == 'AUT'), 'Country'] = 'Austria'
    df.loc[(df['Country'] == 'ZAF'), 'Country'] = 'South Africa'
    df.loc[(df['Country'] == 'NZL'), 'Country'] = 'New Zealand'
    df.loc[(df['Country'] == 'ISR'), 'Country'] = 'Israel'
    df.loc[(df['Country'] == 'HUN'), 'Country'] = 'Hungary'
    df.loc[(df['Country'] == 'GRC'), 'Country'] = 'Greece'
    df.loc[(df['Country'] == 'ROM'), 'Country'] = 'Romania'
    df.loc[(df['Country'] == 'MEX'), 'Country'] = 'Mexico'
    df.loc[(df['Country'] == 'CAN'), 'Country'] = 'Canada'
    df.loc[(df['Country'] == 'TUR'), 'Country'] = 'Turkey'
    df.loc[(df['Country'] == 'CZE'), 'Country'] = 'Czech Republic'
    df.loc[(df['Country'] == 'SVN'), 'Country'] = 'Slovenia'
    df.loc[(df['Country'] == 'LUX'), 'Country'] = 'Luxembourg'
    df.loc[(df['Country'] == 'HRV'), 'Country'] = 'Croatia'
    df.loc[(df['Country'] == 'GEO'), 'Country'] = 'Georgia'
    df.loc[(df['Country'] == 'URY'), 'Country'] = 'Uruguay'
    df.loc[(df['Country'] == 'GBR'), 'Country'] = 'England'
    df.loc[(df['Country'] == 'LBN'), 'Country'] = 'Lebanon'
    df.loc[(df['Country'] == 'YUG'), 'Country'] = 'Serbia'
    df.loc[(df['Country'] == 'BRA'), 'Country'] = 'Brazil'
    df.loc[(df['Country'] == 'MDA'), 'Country'] = 'Moldova'
    df.loc[(df['Country'] == 'MAR'), 'Country'] = 'Morocco'
    df.loc[(df['Country'] == 'PER'), 'Country'] = 'Peru'
    df.loc[(df['Country'] == 'IND'), 'Country'] = 'India'
    df.loc[(df['Country'] == 'BGR'), 'Country'] = 'Bulgaria'
    df.loc[(df['Country'] == 'CYP'), 'Country'] = 'Cyprus'
    df.loc[(df['Country'] == 'ARM'), 'Country'] = 'Armenia'
    df.loc[(df['Country'] == 'CHE'), 'Country'] = 'Switzerland'
    df.loc[(df['Country'] == 'BIH'), 'Country'] = 'Bosnia and Herzegovina'
    df.loc[(df['Country'] == 'UKR'), 'Country'] = 'Ukraine'
    df.loc[(df['Country'] == 'SVK'), 'Country'] = 'Slovakia'
    df.loc[(df['Country'] == 'MKD'), 'Country'] = 'Macedonia'
    df.loc[(df['Country'] == 'CHN'), 'Country'] = 'China'
    df.loc[(df['Country'] == 'EGY'), 'Country'] = 'Egypt'
    return df


In [109]:
wine_base.country.unique()

array(['Italy', 'Portugal', 'US', 'Spain', 'France', 'Germany',
       'Argentina', 'Chile', 'Australia', 'Austria', 'South Africa',
       'New Zealand'], dtype=object)

In [110]:
#parse the right name
df_temp=state_sing(df_temp)
df_rain=state_sing(df_rain)

In [111]:
#check if there all all nations
[x for x in wine_base['country'].unique() if x not in df_rain['Country'].unique()]

[]

In [112]:
import re
def get_vintage(df):
    pattern = r"\d{4}"
    vintages = []
    for elem in df.title:
        match = re.findall(pattern,elem)
        year = 0
        if len(match)>1:
            y= int(match[0])
            if y <1952:
                year = 0
            elif y==3000:
                year == 2009
            elif y == 7200:
                year = int(match[1])
            else:
                year= y
        elif len(match)==1:
            y= int(match[0])
            if y <1952:
                year = 0
            elif y==3000:
                year == 2009
            elif year == 7200:
                year = 0
            else:
                year = y
        else:
            year = 0
        vintages.append(year)

    se = pd.Series(vintages)
    df['vintage']=se.values 
    df.drop('title',axis=1)
    return df

In [113]:
wine_base = get_vintage(wine_base)
#drop reviwe of wine with ventage =0
wine_base = wine_base[wine_base.vintage != 0]
wine_base = wine_base[wine_base.vintage != 2017]
wine_base = wine_base[wine_base.vintage != 2016]

#delete wine where there is no wheter info

In [114]:
wine_base.head(1)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,tfIdf_grouped_1,tfIdf_grouped_2,tfIdf_grouped_3,tf_fullData_1,tf_fullData_2,tf_fullData_3,tfIdf_fullData_1,tfIdf_fullData_2,tfIdf_fullData_3,vintage
0,Italy,tropic broom brimston herb express appl citru ...,Vulkà Bianco,87,17.0,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,...,0.384445,0.305772,0.048858,0.018182,0.005882,0.00064,0.06055,0.016707,0.001205,2013


In [115]:
#take only the correct month and the year that are in the wine dataset 
year_in = wine_base['vintage'].unique() 
df_rain_filtered = df_rain[(df_rain['Year'].isin(year_in)) & (df_rain['Month'].isin([5,6,7,8,9])) ]
df_temp_filtered = df_temp[(df_temp['Year'].isin(year_in)) & (df_temp['Month'].isin([5,6,7,8,9])) ]


In [116]:
#to be fast
wine_base = wine_base.set_index(['vintage','country'])
wine_base= wine_base.sort_index()


In [117]:
wine_base['pr_5']=0
wine_base['pr_6']=0
wine_base['pr_7']=0
wine_base['pr_8']=0
wine_base['pr_9']=0

wine_base['tas_5']=0
wine_base['tas_6']=0
wine_base['tas_7']=0
wine_base['tas_8']=0
wine_base['tas_9']=0

In [118]:
#wine_base = wine_base.set_index(['vintage','country'])
wine_base= wine_base.sort_index()

wine_base.head(2)

description  \
vintage country                                                       
1952    Portugal  year ii oak deep oldgold color rich sweet wood...   
        Portugal  astonish age rich toffe warm intens concentr d...   

                                          designation  points  price province  \
vintage country                                                                 
1952    Portugal  Porto Envelhecido em Casco Colheita      95  499.0     Port   
        Portugal                       Colheita Tawny      96  415.0     Port   

                 region_1 taster_name  \
vintage country                         
1952    Portugal  unknown  Roger Voss   
        Portugal  unknown  Roger Voss   

                                                              title variety  \
vintage country                                                               
1952    Portugal  Messias 1952 Porto Envelhecido em Casco Colhei...    Port   
        Portugal              Burmester 1952 Colheita Tawny  (Port)    Port   

                     winery  ...    pr_5  pr_6  pr_7  pr_8  pr_9  tas_5  \
vintage country              ...                                          
1952    Portugal    Messias  ...       0     0     0     0     0      0   
        Portugal  Burmester  ...       0     0     0     0     0      0   

                  tas_6  tas_7  tas_8  tas_9  
vintage country                               
1952    Portugal      0      0      0      0  
        Portugal      0      0      0      0  

[2 rows x 34 columns]

In [119]:
for index, row in df_rain_filtered.iterrows(): 
    try:
        wine_base.loc[[[row['Year'],row['Country']]],'pr_'+str(row['Month'])] = row['pr']
        pass
    except KeyError:
        #if there is no wine in that cantry in that year
        pass

In [120]:
for index, row in df_temp_filtered.iterrows(): 
    try:
        wine_base.loc[[[row['Year'],row['Country']]],'tas_'+str(row['Month'])] = row['tas']
        pass
    except KeyError:
        #if there is no wine in that cantry in that year
        pass

In [121]:
df_temp_filtered.head(1)

,tas,Year,Month,Country
616,11.7139,1952,5,Argentina


In [122]:
wine_base = wine_base.reset_index()
wine_base.head(2)

,vintage,country,description,designation,points,price,province,region_1,taster_name,title,...,pr_5,pr_6,pr_7,pr_8,pr_9,tas_5,tas_6,tas_7,tas_8,tas_9
0,1952,Portugal,year ii oak deep oldgold color rich sweet wood...,Porto Envelhecido em Casco Colheita,95,499.0,Port,unknown,Roger Voss,Messias 1952 Porto Envelhecido em Casco Colhei...,...,120.181,54.8968,13.3656,12.3677,71.6485,16.0018,20.1771,21.6355,21.3967,18.5723
1,1952,Portugal,astonish age rich toffe warm intens concentr d...,Colheita Tawny,96,415.0,Port,unknown,Roger Voss,Burmester 1952 Colheita Tawny (Port),...,120.181,54.8968,13.3656,12.3677,71.6485,16.0018,20.1771,21.6355,21.3967,18.5723


In [123]:
list(wine_base)

['vintage',
 'country',
 'description',
 'designation',
 'points',
 'price',
 'province',
 'region_1',
 'taster_name',
 'title',
 'variety',
 'winery',
 'similarityTop3WinesByVariety',
 'word_count',
 'tf_grouped_1',
 'tf_grouped_2',
 'tf_grouped_3',
 'tfIdf_grouped_1',
 'tfIdf_grouped_2',
 'tfIdf_grouped_3',
 'tf_fullData_1',
 'tf_fullData_2',
 'tf_fullData_3',
 'tfIdf_fullData_1',
 'tfIdf_fullData_2',
 'tfIdf_fullData_3',
 'pr_5',
 'pr_6',
 'pr_7',
 'pr_8',
 'pr_9',
 'tas_5',
 'tas_6',
 'tas_7',
 'tas_8',
 'tas_9']

In [124]:
wine_base= wine_base.drop(['designation','title'], axis=1)

# encode it

In [125]:
#label each colum that you want to use

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

wine_base["province"] = wine_base["province"].astype('category').cat.codes
wine_base["country"] = wine_base["country"].astype('category').cat.codes
wine_base["taster_name"] = wine_base["taster_name"].astype('category').cat.codes
wine_base["winery"] = wine_base["winery"].astype('category').cat.codes
wine_base["region_1"] = wine_base["region_1"].astype('category').cat.codes
wine_base["variety"] = wine_base["variety"].astype('category').cat.codes




In [126]:
wine_base.head(1)

,vintage,country,description,points,price,province,region_1,taster_name,variety,winery,...,pr_5,pr_6,pr_7,pr_8,pr_9,tas_5,tas_6,tas_7,tas_8,tas_9
0,1952,8,year ii oak deep oldgold color rich sweet wood...,95,499.0,164,1201,13,376,10276,...,120.181,54.8968,13.3656,12.3677,71.6485,16.0018,20.1771,21.6355,21.3967,18.5723


In [127]:
wine_base.to_csv("StemmedWord2vecTop3_parsed_weather_labeled.csv", encoding='utf-8', index=False)


In [129]:
wine_base.head(50)

,vintage,country,description,points,price,province,region_1,taster_name,variety,winery,...,pr_5,pr_6,pr_7,pr_8,pr_9,tas_5,tas_6,tas_7,tas_8,tas_9
0,1952,8,year ii oak deep oldgold color rich sweet wood...,95,499.0,164,1201,13,376,10276,...,120.1810,54.8968,13.36560,12.36770,71.64850,16.0018,20.1771,21.6355,21.3967,18.5723
1,1952,8,astonish age rich toffe warm intens concentr d...,96,415.0,164,1201,13,376,1703,...,120.1810,54.8968,13.36560,12.36770,71.64850,16.0018,20.1771,21.6355,21.3967,18.5723
2,1957,8,great balanc fresh age vener port cours mani w...,95,375.0,164,1201,13,376,8743,...,92.0486,44.2920,6.72316,7.40562,49.36750,15.0959,18.7823,22.4479,22.4856,20.6839
3,1961,8,old gold color matur colheita great vintag por...,95,320.0,164,1201,13,376,5078,...,90.5883,43.2681,25.67690,2.98664,43.91890,18.0467,20.0059,21.9887,23.7509,20.2514
4,1963,8,matur port past year rich fig chocol touch alm...,95,467.0,164,1201,17,376,6238,...,38.1876,48.3052,3.52590,4.78859,29.14870,16.1193,18.4244,22.1413,20.8027,19.1788
5,1963,8,fabul legendari vintag port year age wood conc...,96,400.0,164,1201,13,376,10276,...,38.1876,48.3052,3.52590,4.78859,29.14870,16.1193,18.4244,22.1413,20.8027,19.1788
6,1963,8,sweet soft age light wood spice edg spirit fre...,87,790.0,164,1201,13,376,1703,...,38.1876,48.3052,3.52590,4.78859,29.14870,16.1193,18.4244,22.1413,20.8027,19.1788
7,1964,8,toffe vanilla sweet molass caramel consist hon...,91,125.0,164,1201,13,376,15321,...,22.0317,56.3338,8.25345,5.69519,38.65200,18.4907,19.7043,22.1509,22.5522,21.9118
8,1964,8,old gold color singl year cask wonder velvet t...,95,588.0,164,1201,13,376,13756,...,22.0317,56.3338,8.25345,5.69519,38.65200,18.4907,19.7043,22.1509,22.5522,21.9118
9,1965,8,year singl quinta vintag port concentr toffe r...,94,240.0,164,1201,13,376,8743,...,17.4721,16.5924,4.11930,3.63757,111.79200,18.1501,20.2796,20.0903,22.2992,17.8837
